In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer

from scipy import stats
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

import statsmodels.api as sm
from linearmodels.panel import PooledOLS

In [3]:
df = pd.read_pickle("data_after_normalization-v1.pickle")

In [4]:
df = df.drop(['Charging_Time_in_hours', 'Precipitation','Temperature','pd','Bahnhof','Restaurant'
              ,'Biergarten','Community_Centre','Hospital','Kino','Parking','university','Charging_time_in_hours_log_New'], axis = 1)

In [5]:
df = df.rename(columns = {'Temperature_New':'Temperature', 'Precipitation_New':'Precipitation',
                          'pd_New':'pd','Bahnhof_New':'Bahnhof','Restaurant_New':'Restaurant','Biergarten_New':'Biergarten',
                          'Community_Centre_New':'Community_Centre', 'Hospital_New':'Hospital',
                         'Kino_New':'Kino', 'Parking_New':'Parking','university_New':'university', 'Charging_Time_in_hours_New':'Charging_Time_in_hours',
                          })

In [6]:
df['Year'] = pd.to_datetime(df['datetime_start'], format='%Y')

In [7]:
df['Month'] = pd.to_datetime(df['datetime_start'], format='%M')

In [8]:
df['Day'] = pd.to_datetime(df['datetime_start'], format='%D')

In [9]:
data = df.copy()

In [10]:
data

,park_id,clc_code,area_code,evse_id,datetime_start,datetime_stop,city,latitude,longitude,postal_code,...,Biergarten,Community_Centre,Hospital,Kino,Parking,university,Charging_Time_in_hours,Year,Month,Day
0,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:00:13.058,Kremmen,52.7596,13.0239,16766,...,0.261462,0.083498,20.0,20.000000,0.352947,20.000000,20.000000,2019-12-17 09:00:13.369,2019-12-17 09:00:13.369,2019-12-17 09:00:13.369
1,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:01:16.290,Kremmen,52.7596,13.0239,16766,...,0.261462,0.083498,20.0,20.000000,0.352947,20.000000,0.014352,2019-12-17 09:00:13.369,2019-12-17 09:00:13.369,2019-12-17 09:00:13.369
2,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:58:30.703,Kremmen,52.7596,13.0239,16766,...,0.261462,0.083498,20.0,20.000000,0.352947,20.000000,0.809500,2019-12-17 09:00:13.369,2019-12-17 09:00:13.369,2019-12-17 09:00:13.369
3,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:58:28.352,2019-12-17 09:00:13.058,Kremmen,52.7596,13.0239,16766,...,0.261462,0.083498,20.0,20.000000,0.352947,20.000000,19.190963,2019-12-17 09:58:28.352,2019-12-17 09:58:28.352,2019-12-17 09:58:28.352
4,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:58:28.352,2019-12-17 09:01:16.290,Kremmen,52.7596,13.0239,16766,...,0.261462,0.083498,20.0,20.000000,0.352947,20.000000,19.205546,2019-12-17 09:58:28.352,2019-12-17 09:58:28.352,2019-12-17 09:58:28.352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940929,sl_23897386,Urban,051120000000,sl_23901984,2021-08-07 19:03:56.000,2021-08-07 16:02:00.000,Niestetal,51.3144,9.5549,34266,...,0.037194,0.042186,20.0,0.078859,0.062891,0.072758,17.473350,2021-08-07 19:03:56.000,2021-08-07 19:03:56.000,2021-08-07 19:03:56.000
1940930,sl_23897386,Urban,051120000000,sl_23901984,2021-08-07 19:03:56.000,2021-08-07 16:28:29.000,Niestetal,51.3144,9.5549,34266,...,0.037194,0.042186,20.0,0.078859,0.062891,0.072758,17.841179,2021-08-07 19:03:56.000,2021-08-07 19:03:56.000,2021-08-07 19:03:56.000
1940931,sl_23897386,Urban,051120000000,sl_23901984,2021-08-07 19:03:56.000,2021-08-07 19:55:30.000,Niestetal,51.3144,9.5549,34266,...,0.037194,0.042186,20.0,0.078859,0.062891,0.072758,0.716212,2021-08-07 19:03:56.000,2021-08-07 19:03:56.000,2021-08-07 19:03:56.000
1940932,sl_23897386,Urban,051120000000,sl_23901984,2021-08-10 15:29:47.000,2021-08-10 16:57:44.000,Niestetal,51.3144,9.5549,34266,...,0.037194,0.042186,20.0,0.078859,0.062891,0.072758,1.221542,2021-08-10 15:29:47.000,2021-08-10 15:29:47.000,2021-08-10 15:29:47.000


In [11]:
from linearmodels import PanelOLS
import pandas as pd

In [12]:
mod = PanelOLS.from_formula('Charging_time_in_hours_log ~ clc_code + month + day_of_week_name_Start +pd+Precipitation+Temperature+Parking+university+Biergarten+Kino+Restaurant+Community_Centre+Hospital+Bahnhof + EntityEffects+year',data=data.set_index(['park_id', 'Year']),drop_absorbed=True)

/home/col-gku/.local/lib/python3.8/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [13]:
result = mod.fit(cov_type='clustered', cluster_entity=True)

/home/col-gku/.local/lib/python3.8/site-packages/linearmodels/panel/model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

clc_code[Suburban], clc_code[Uninhabited], clc_code[Urban], pd

  warnings.warn(


In [14]:
#result.estimated_effects.to_csv("estimated_effects.csv")

In [15]:
l = result.estimated_effects

In [16]:
print((result.summary).as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                          & Charging\_time\_in\_hours\_log & \textbf{  R-squared:         }   &      0.0070      \\
\textbf{Estimator:}                              &            PanelOLS            & \textbf{  R-squared (Between):}  &     -22.936      \\
\textbf{No. Observations:}                       &            1796283             & \textbf{  R-squared (Within):}   &      0.0070      \\
\textbf{Date:}                                   &        Wed, Jan 26 2022        & \textbf{  R-squared (Overall):}  &     -2.3136      \\
\textbf{Time:}                                   &            12:14:59            & \textbf{  Log-likelihood     }   &    -3.607e+06    \\
\textbf{Cov. Estimator:}                         &           Clustered            & \textbf{                     }   &                  \\
\textbf{}                                        &                                & \textbf{  F-statistic:       }   &  

In [1]:
#import sys
#filename  = open("panel_ols_with_entity_effexts",'w')
#sys.stdout = filename
#print(result)